### 프로젝트 개발 시작

# 라이브러리 선언

In [7]:
# 1. 라이브러리 선언
import pandas as pd
import numpy as np

# 데이터 불러오기

In [8]:
# 2. 데이터 불러 오기
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

#3-1. 데이터 살펴보기
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [9]:
#3-2 데이터 타입 살펴보기
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
dtype: object

# 데이터 정제 하기

### 1) 음수 데이터 정제 하기

In [10]:
#음수 데이터 삺펴보기
selloutData[(selloutData["QTY"]<0)].head(5)

,REGIONID,PRODUCT,YEARWEEK,QTY
65,A02,PRODUCT16,201403,-1.0
338,A08,PRODUCT36,201402,-1.0
1270,A14,PRODUCT36,201405,-1.0
1616,A17,PRODUCT60,201402,-1.0
2035,A39,PRODUCT7,201404,-12.0


In [11]:
#데이터 정제 하기
# 함수 정의
def refining(df):
    if df['QTY'] < 0:
        return 0
    else:
        return df['QTY']

# 음수 데이터 정제    
selloutData["QTY_NEW"] = selloutData.apply(refining, axis=1)
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0


In [15]:
# 음수 데이터 여부 확인 하기 
selloutData[(selloutData.QTY_NEW < 0)]

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW


### 53주차 데이터 정제 하기

In [21]:
# 스트링 타입 변경 하기 
selloutData["YEARWEEK"] = selloutData["YEARWEEK"].astype(str)

# 53 주차 데이터 제외하고생성
selloutData_V2 = selloutData[(selloutData.YEARWEEK.str[4:6] != "53")]

# 53 주체 데이터 존재 하는지 검증
selloutData_V2[(selloutData_V2.YEARWEEK.str[4:6] == "53")]

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW


In [22]:
selloutData_V2.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0


## WEEK 데이터 분리 하기

In [24]:
#4-3 WEEK 데이터 분리 하기
selloutData_V2['WEEK'] = selloutData_V2.YEARWEEK.str[4:6]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# 이동 평균/표준편차 데이터 만들기

In [25]:
#데이터 정렬
selloutData_V2.sort_values(["REGIONID","PRODUCT","YEARWEEK"], ascending=[True,True,True], inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
#인덱스 초기화
selloutData_V2 = selloutData_V2.reset_index()
selloutData_V2.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,WEEK
0,298,A00,PRODUCT34,201401,661.0,661.0,01
1,1757,A00,PRODUCT34,201402,679.0,679.0,02
2,3125,A00,PRODUCT34,201403,578.0,578.0,03
3,205,A00,PRODUCT34,201404,532.0,532.0,04
4,4369,A00,PRODUCT34,201405,516.0,516.0,05


### 이동 평균 데이터 구하기

In [27]:
#그룹바이해서 롤링(이동평균) -
# 이동평균 함수 생성합
def sub_function(data):
    data["MA"] = data["QTY_NEW"].rolling(window = 13, center = True, min_periods=1).mean()
    return data

# 그룹바이 + 이동 평균 생성
selloutData_V2 = selloutData_V2.groupby(["REGIONID","PRODUCT"]).apply(sub_function)

In [28]:
selloutData_V2.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,WEEK,MA
0,298,A00,PRODUCT34,201401,661.0,661.0,01,542.285714
1,1757,A00,PRODUCT34,201402,679.0,679.0,02,520.000000
2,3125,A00,PRODUCT34,201403,578.0,578.0,03,514.444444
3,205,A00,PRODUCT34,201404,532.0,532.0,04,516.800000
4,4369,A00,PRODUCT34,201405,516.0,516.0,05,503.363636


## 표준편차 구하기

In [29]:
# 표준편차 함수 생성
def sub_function2(data):
    data["STD"] = data["MA"].rolling(window = 5, center = True, min_periods=1).std()
    return data

selloutData_V2 = selloutData_V2.groupby(["REGIONID","PRODUCT"]).apply(sub_function2)

In [30]:
selloutData_V2.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,WEEK,MA,STD
0,298,A00,PRODUCT34,201401,661.0,661.0,01,542.285714,14.734617
1,1757,A00,PRODUCT34,201402,679.0,679.0,02,520.000000,12.806131
2,3125,A00,PRODUCT34,201403,578.0,578.0,03,514.444444,14.253036
3,205,A00,PRODUCT34,201404,532.0,532.0,04,516.800000,12.002994
4,4369,A00,PRODUCT34,201405,516.0,516.0,05,503.363636,15.591135


## 상/하한선 데이터 생성

In [31]:
# 상/하한선 작성
selloutData_V2["UPPER_BOUND"] = selloutData_V2["MA"] + selloutData_V2["STD"]
selloutData_V2["LOWER_BOUND"] = selloutData_V2["MA"] - selloutData_V2["STD"]

selloutData_V2.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,WEEK,MA,STD,UPPER_BOUND,LOWER_BOUND
0,298,A00,PRODUCT34,201401,661.0,661.0,01,542.285714,14.734617,557.020331,527.551098
1,1757,A00,PRODUCT34,201402,679.0,679.0,02,520.000000,12.806131,532.806131,507.193869
2,3125,A00,PRODUCT34,201403,578.0,578.0,03,514.444444,14.253036,528.697481,500.191408
3,205,A00,PRODUCT34,201404,532.0,532.0,04,516.800000,12.002994,528.802994,504.797006
4,4369,A00,PRODUCT34,201405,516.0,516.0,05,503.363636,15.591135,518.954771,487.772501


### 정제된 판매량 산출

In [34]:
#정제된 판매량 산출
selloutData_V2["QTY_REFINED"] = np.where(selloutData_V2["QTY_NEW"]>selloutData_V2["UPPER_BOUND"],selloutData_V2["UPPER_BOUND"],
                                         np.where(selloutData_V2["QTY_NEW"]<selloutData_V2["LOWER_BOUND"],selloutData_V2["LOWER_BOUND"],selloutData_V2["QTY_NEW"]))

In [35]:
selloutData_V2.head()

,index,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,WEEK,MA,STD,UPPER_BOUND,LOWER_BOUND,QTY_REFINED
0,298,A00,PRODUCT34,201401,661.0,661.0,01,542.285714,14.734617,557.020331,527.551098,557.020331
1,1757,A00,PRODUCT34,201402,679.0,679.0,02,520.000000,12.806131,532.806131,507.193869,532.806131
2,3125,A00,PRODUCT34,201403,578.0,578.0,03,514.444444,14.253036,528.697481,500.191408,528.697481
3,205,A00,PRODUCT34,201404,532.0,532.0,04,516.800000,12.002994,528.802994,504.797006,528.802994
4,4369,A00,PRODUCT34,201405,516.0,516.0,05,503.363636,15.591135,518.954771,487.772501,516.000000


## 스무딩 작성

In [36]:
#스무딩 작성 
def sub_function2(data):
    data["SMOOTHING"] = data["QTY_REFINED"].rolling(window = 5, center = True, min_periods=1).mean()
    return data

selloutData_V2 = selloutData_V2.groupby(["REGIONID","PRODUCT"]).apply(sub_function2)

In [130]:
selloutData_V2.head(2)

,index,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,WEEK,MA,STD,UPPER_BOUND,LOWER_BOUND,QTY_REFINED,SMOOTHING
0,298,A00,PRODUCT34,201401,661.0,661.0,01,542.285714,14.734617,557.020331,527.551098,557.020331,539.507981
1,1757,A00,PRODUCT34,201402,679.0,679.0,02,520.000000,12.806131,532.806131,507.193869,532.806131,536.831734


## 계절성 지수 산출

In [37]:
#계절성 지수 산정
#안정적
selloutData_V2["STABLE"] = selloutData_V2["QTY_NEW"] / selloutData_V2["SMOOTHING"]
#불안정
selloutData_V2["UNSTABLE"] = selloutData_V2["QTY_REFINED"] / selloutData_V2["SMOOTHING"]

In [38]:
selloutData_V2.head(10)

,index,REGIONID,PRODUCT,YEARWEEK,QTY,QTY_NEW,WEEK,MA,STD,UPPER_BOUND,LOWER_BOUND,QTY_REFINED,SMOOTHING,STABLE,UNSTABLE
0,298,A00,PRODUCT34,201401,661.0,661.0,01,542.285714,14.734617,557.020331,527.551098,557.020331,539.507981,1.225190,1.032460
1,1757,A00,PRODUCT34,201402,679.0,679.0,02,520.000000,12.806131,532.806131,507.193869,532.806131,536.831734,1.264828,0.992501
2,3125,A00,PRODUCT34,201403,578.0,578.0,03,514.444444,14.253036,528.697481,500.191408,528.697481,532.665387,1.085109,0.992551
3,205,A00,PRODUCT34,201404,532.0,532.0,04,516.800000,12.002994,528.802994,504.797006,528.802994,514.586876,1.033839,1.027626
4,4369,A00,PRODUCT34,201405,516.0,516.0,05,503.363636,15.591135,518.954771,487.772501,516.000000,497.704610,1.036760,1.036760
5,4313,A00,PRODUCT34,201406,423.0,423.0,06,490.750000,24.122226,514.872226,466.627774,466.627774,475.126508,0.890289,0.982113
6,6288,A00,PRODUCT34,201407,407.0,407.0,07,480.230769,31.835969,512.066738,448.394800,448.394800,462.229081,0.880516,0.970071
7,6403,A00,PRODUCT34,201408,364.0,364.0,08,453.384615,37.577643,490.962258,415.806973,415.806973,447.085420,0.814162,0.930039
8,5158,A00,PRODUCT34,201409,470.0,470.0,09,423.846154,40.469705,464.315859,383.376449,464.315859,427.559865,1.099261,1.085967
9,7437,A00,PRODUCT34,201410,538.0,538.0,10,401.230769,39.050923,440.281692,362.179847,440.281692,408.280905,1.317720,1.078379
